# Sphere morphing


In [ ]:
%load_ext autoreload

import numpy as np
import random
import torch
import pickle

from ipywidgets import interact 
import gc
import open3d as o3d

from src.visualisation import *
from src.chamfer import *
from src.utils import *
from src.morph import *

random.seed = 42


### sphere morphing

visualise the morphing of a sphere into a shape as an animation.


In [ ]:
%autoreload 2

# visualise the morphing of a sphere into a shape
cld1_name = "data/plane1.pcd"
visualise = True
#loss_funcs = [ "chamfer", "emd", "uniform", "reverse", "single"]
#loss_funcs = [ "uniform", "single"]
loss_funcs = [ "density"]
for loss_func in loss_funcs:
    print(loss_func)
    run_morph(cld1_name, loss_func)
    if visualise:
        with open("data/" + loss_func + ".pkl", "rb") as f:
            morphed = pickle.load(f)
        colours = visualise_density(morphed, 'plasma_r')
        with open("data/" + loss_func + "_dens.pkl", "wb") as f:
            pickle.dump(colours, f)
        #create_point_cloud_animation(cloud_list, loss_func)
    torch.cuda.empty_cache()
    gc.collect()

In [ ]:
def view_density(loss_func): 
    with open("data/" + loss_func + ".pkl", "rb") as f:
        morphed = pickle.load(f)
    with open("data/" + loss_func + "_dens.pkl", "rb") as f:
        colours = pickle.load(f)

    create_point_cloud_animation(morphed, loss_func, True, colours[:,:,:3])
    
interact(view_density, loss_func=[ "density", "uniform", "infocd", "single", "chamfer", "reverse", "emd", "direct"]); 


In [ ]:
torch.cuda.empty_cache()
gc.collect()

# visualise animation
loss_func = "emd"
with open("data/" + loss_func + ".pkl", "rb") as f:
    morphed = pickle.load(f)
print(morphed.shape, loss_func)
cloud_list = [m for m in morphed]
#create_point_cloud_animation(cloud_list, loss_func)


In [ ]:
colours = visualise_density(morphed, 'plasma_r')
with open("data/" + loss_func + "_dens.pkl", "wb") as f:
    pickle.dump(colours, f)

### Batch optimisation

Perform sphere morphing in batches on entire PCN testset to generate output metrics


In [ ]:
# downsample
shapenet_path = "../experiments/ICCV2023-HyperCD/ShapeNetCompletion/test/complete/"
downsampled_path = "../experiments/ICCV2023-HyperCD/ShapeNetCompletion/downsample/"
downsample = False
if downsample:
    folders = os.listdir(shapenet_path)
    

    for fl in tqdm(folders):
        if not os.path.exists(downsampled_path+fl):
            os.mkdir(downsampled_path+fl)
            
        files = os.listdir(shapenet_path + fl)
        cloud =  o3d.geometry.PointCloud()
        for cl in files:
            points = np.array(o3d.io.read_point_cloud(shapenet_path + fl + "/" + cl).points)
            choices = np.random.choice(len(points), 4096)
            points = points[choices]
            cloud.points = o3d.utility.Vector3dVector(points)
            o3d.io.write_point_cloud(downsampled_path+fl + "/" + cl, cloud)

In [ ]:
# calculate metrics and return cd and emd values only
loss_func = "uniform"
cd, emd = sphere_morph_metrics(loss_func, downsampled_path, save=False)
print(cd[-1], emd[-1])

In [ ]:
# calculate values and save cd, emd, and point assignments
loss_func = "uniform"

sphere_morph_metrics(loss_func, downsampled_path)

In [ ]:
# plot losses on same axis
def plot_losses(losses, labels, title):
    x = np.arange(0, len(losses[0]))
    plt.figure(figsize=(30, 6))
    for i, loss in enumerate(losses):
        plt.plot(x, loss, label=labels[i])

    plt.xlabel("point cloud index")
    plt.ylabel("distance")
    plt.title(title)
    plt.legend()
    plt.show()

In [ ]:
# create plots
loss_funcs = ["chamfer",]
chamfer_list, emd_list = [], []
for loss_func in loss_funcs:
    with open("data/" + loss_func + "_metrics.pkl", "rb") as f:
        chamfer, emd, assignments = pickle.load(f)
        chamfer_list.append(chamfer)
        emd_list.append(emd)

plot_losses(chamfer_list, loss_funcs, "chamfer")

plot_losses(emd_list, loss_funcs, "EMD")

print(emd_list[-1], chamfer_list[-1])

In [ ]:
# load losses
loss_types = ["reverse", "chamfer", "emd", "pair"]
losses = []

for loss_func in loss_types:
    with open("data/loss_" + loss_func + ".pkl", "rb") as f:
        losses.append(pickle.load(f))

plot_losses(losses, loss_types, "loss function comparison")


#### consistency

Consistency measurements for a single cloud. Refer to loss1.ipynb for batch metrics.


In [ ]:
# measure conssistency between forward and backward correspondences for chamfer distance
# optionally compare against the ideal assignment, as measured by EMD
def measure_assignment_consistency(assignment, emd=None):
    reverse_assignment = torch.gather(assignment[0], 0, assignment[1])
    expected = torch.arange(assignment[0].shape[0], device=torch.device("cuda"))
    consistency = torch.sum(torch.eq(expected, reverse_assignment).long())
    print("consistency", consistency.item(), len(torch.unique(assignment[0])), len(torch.unique(assignment[1])))
    
    if emd is not None:
        #print(emd[:5], assignment[0][:5], assignment[1][:5])
        emd_consistency = torch.sum(torch.eq(emd, assignment[0]).long())
        #print("emd_consistency", emd_consistency.item(), len(torch.unique(emd)))
